LLM控制面板

In [1]:
from model import *
from IPython.display import display

model_list = [
        ("gpt2", "/home/cs/yangyuchen/guoyiqiu/my_models/gpt2"),
        ("llama_7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-7b-hf"),
        ("llama_13b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-13b"),
        ("llama-2-7b-chat", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-2-7b-chat-hugging"),
        ("llama-2-13b-chat", "/home/cs/yangyuchen/guoyiqiu/my_models/Llama-2-13b-chat-ms"),
        ("vicuna_7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/vicuna-7b/"),
        ("internlm-chat-7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/internlm-chat-7b"),
        ("internlm-chat-20b", "/home/cs/yangyuchen/guoyiqiu/my_models/internlm-chat-20b"),
    ]

panel = LLMPanel(model_list, chat_template=INTERNLM_TEMPLATE)
display(panel)

LLMPanel(children=(HBox(children=(Dropdown(description='Model:', options=(('gpt2', '/home/cs/yangyuchen/guoyiq…

Using pad_token, but it is not set yet.


Everything is ready. Time cost: 4.70s


FlowVisualizer

In [ ]:
utokens_tab, entropy_tab = panel.mt.vis_sentence(f'hello world',max_new_tokens=64)
utokens_tab.render_notebook()

In [ ]:
import json
import jsonlines

usmle_test = list(jsonlines.open("data/usmle/questions/US/test.jsonl"))
usmle_test_for_infer = []

for d in usmle_test:
    options_strs = [f"{op}: {d['options'][op]}" for op in d['options']]
    input = f"Question: {d['question']} Options: {'; '.join(options_strs)}. Output: The correct answer is option"
    input = INTERNLM_TEMPLATE.format(input)
    d['input'] = input
    d['labels'] = ['A', 'B', 'C', 'D', 'E']
    usmle_test_for_infer.append(d)

json.dump(usmle_test_for_infer, open("data/mgpu_infer/usmle_test_inference.json", "w"), indent=4)

MultiGPU Inference

In [3]:
from model.llm_utils import get_free_gpus
import subprocess

free_gpus = get_free_gpus()
command = f'CUDA_VISIBLE_DEVICES={",".join([str(i) for i in free_gpus])} \
            torchrun --nproc_per_node {len(free_gpus)} --master_port=12570 /home/cs/yangyuchen/guoyiqiu/gpt_re/mgpu_infer.py \
            --func gen \
            --model_path /home/cs/yangyuchen/yushengliao/Medical_LLM/vicuna-7b/ \
            --dst_path /home/cs/yangyuchen/guoyiqiu/gpt_re/data/mgpu_infer/usmle_test_inference.json \
            --save_path /home/cs/yangyuchen/guoyiqiu/gpt_re/data/mgpu_infer_output/vicuna-7b_usmle_test_generate.json \
            --mnt 8'

process = subprocess.Popen(command, shell=True)
print(f"shell process id: {process.pid}")

shell process id: 2737327


[2023-10-16 17:29:17,296] torch.distributed.run: [WARNING] 
[2023-10-16 17:29:17,296] torch.distributed.run: [WARNING] *****************************************
[2023-10-16 17:29:17,296] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2023-10-16 17:29:17,296] torch.distributed.run: [WARNING] *****************************************


model_path: /home/cs/yangyuchen/yushengliao/Medical_LLM/vicuna-7b/
 dst_path: /home/cs/yangyuchen/guoyiqiu/gpt_re/data/mgpu_infer/usmle_test_inference.json
 save_path: /home/cs/yangyuchen/guoyiqiu/gpt_re/data/mgpu_infer_output/vicuna-7b_usmle_test_generate.json
 func: multigpu_generate
 seed: 42
 kwargs: {'mnt': 8}
 world_size: 8
 CUDA_VISIBLE_DEVICES: 0,1,2,3,4,5,6,7



Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.22s/it]
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.24s/it]
Using pad_token, but it is not set yet.
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.38s/it]
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


dst size:  1273


  0%|          | 0/159 [00:00<?, ?it/s]/home/cs/yangyuchen/miniconda3/envs/med_llm_gyq/lib/python3.9/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
  0%|          | 0/159 [00:00<?, ?it/s]/home/cs/yangyuchen/miniconda3/envs/med_llm_gyq/lib/python3.9/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-t

rank 0 waiting for collecting results...


100%|██████████| 160/160 [01:07<00:00,  2.35it/s]


all done.


In [ ]:
import json
import numpy as np


internlm_chat_7b_usmle_test_generate = json.load(open('data/mgpu_infer_output/internlm-chat-7b_usmle_test_generate_chat.json'))
internlm_chat_7b_usmle_test_inference = json.load(open('data/mgpu_infer_output/internlm-chat-7b_usmle_test_inference_chat.json'))

inference_acc = 0
generate_acc = 0
i_g = 0
for di,dg in zip(internlm_chat_7b_usmle_test_inference, internlm_chat_7b_usmle_test_generate):
    pred_i = ['A','B','C','D','E'][np.argmin(di['label_loss'])]
    pred_g = dg['output'].replace("<eoa>","").replace("</s>","").strip()[0]
    gt = di['answer_idx']
    inference_acc += (pred_i == gt)
    generate_acc += (pred_g == gt)
    i_g += (pred_i == pred_g)
    
print(f"inference_acc: {inference_acc/len(internlm_chat_7b_usmle_test_inference)}")
print(f"generate_acc: {generate_acc/len(internlm_chat_7b_usmle_test_inference)}")
print(f"i_g: {i_g/len(internlm_chat_7b_usmle_test_inference)}")

LAT

In [ ]:
def pca():
    pass

def make_stimuli_set():
    pass

def lat(mt, stimuli_set, concept):
    pass